 Graniczna Analiza Danych — Raport

## Autorzy
- Daniel Zdancewicz [145317]
- Alex Terentowicz [145419]


## Opis zbiorów danych

Zbiór danych zawierający informacje o lotniskach

[Wejścia](./resources/datasets/inputs.csv):
- i1: roczna przepustowość terminalu zdefiniowana jako przepływ pasażerów, który port lotniczy może obsłużyć bez poważnych niedogodności (w milionach pasażerów rocznie)
- i2: maksymalna przepustowość zdefiniowana jako średnia liczba operacji (przylotów i/lub odlotów), które można wykonać na pasach startowych portu lotniczego (w liczbie operacji na godzinę)
- i3: przepustowość płyty postojowej lotniska definiowana jako średnia liczba samolotów, które może obsłużyć lotnisko (w liczbie samolotów na godzinę)
- i4: obszar ciążenia lotniska zdefiniowany jako liczba mieszkańców mieszkających w promieniu 100 km od lotniska (w mln mieszkańców)

[Wyjścia](./resources/datasets/outputs.csv):
- o1: ruch pasażerski mierzony całkowitą liczbą pasażerów obsłużonych przez port (w mln pasażerów rocznie)
- o2: liczba operacji statku powietrznego (jeden całkowity ruch to lądowanie lub start statku powietrznego; w tysiącach ruchów rocznie)


[Próbki](./resources/datasets/samples.csv):
- i1: waga wejścia i1
- i2: waga wejścia i2
- i3: waga wejścia i3
- i4: waga wejścia i4
- o1: waga wyjścia i1
- o2: waga wyjścia i2


### Odczyt danych

In [8]:
import pandas as pd


def load(path: str):
  return pd.read_csv(path, sep=';')


inputs_ds = load('./resources/datasets/inputs.csv')
outputs_ds = load('./resources/datasets/outputs.csv')
samples_ds = load('./resources/datasets/samples.csv')

In [9]:
inputs_ds.head()

,Unnamed: 0,i1,i2,i3,i4
0,WAW,10.5,36,129.4,7.0
1,KRK,3.1,19,31.6,7.9
2,KAT,3.6,32,57.4,10.5
3,WRO,1.5,12,18.0,3.0
4,POZ,1.5,10,24.0,4.0


In [20]:
outputs_ds.head()

,Unnamed: 0,o1,o2
0,WAW,9.5,129.7
1,KRK,2.9,31.3
2,KAT,2.4,21.1
3,WRO,1.5,18.8
4,POZ,1.3,16.2


In [11]:
samples_ds.head()

,sample,i1,i2,i3,i4,o1,o2
0,1,0.096443,0.059350,0.065224,0.778983,0.620824,0.379176
1,2,0.032029,0.304959,0.427426,0.235586,0.513536,0.486464
2,3,0.266256,0.246940,0.217498,0.269306,0.247490,0.752510
3,4,0.159062,0.635512,0.202159,0.003267,0.223838,0.776162
4,5,0.096716,0.809025,0.026741,0.067518,0.136140,0.863860


## Efektywność

Do wyznaczenia efektywności został utworzony model analizy danych **CCR** zorientowany na nakłady ( wejścia ).

\begin{align*}
\max & \sum_{n=1}^{N}\mu_n\cdot y_{no} & \\
s.t. & \sum\limits_{m=1}^{M}\nu_m\cdot x_{mo}=1  &  \\
 & \sum\limits_{n=1}^{N}\mu_n\cdot y_{nk}\leq\sum\limits_{m=1}^{M}\nu_n\cdot x_{mk}, & k=1,2,...,K \\
 & \mu_n\nu_m\geq0, & m=1,2,...,M\;, n=1,2,...,N \\
\end{align*}



In [123]:
import pulp
from pulp import LpVariable, LpProblem, LpMaximize


def create_model(alternative_nr: int, inputs_df: pd.DataFrame, outputs_df: pd.DataFrame):
  N, M, K = map(len, [outputs_df.columns[1:], inputs_df.columns[1:], inputs_df])
  o = alternative_nr

  model = LpProblem("CCR-input", LpMaximize)

  input_weights = LpVariable.dicts("input_weight", range(M), 0, None, cat='Continuous')
  output_weights = LpVariable.dicts("output_weight", range(N), 0, None, cat='Continuous')

  inputs = inputs_df.iloc[:, 1:].values.T
  outputs = outputs_df.iloc[:, 1:].values.T

  model.setObjective(sum(
    output_weights[n] * outputs[n][o] for n in range(N)
  ))

  model.addConstraint(sum(
    input_weights[m] * inputs[m][o] for m in range(M)
  ) == 1)

  for k in range(K):
    model.addConstraint(
      sum(output_weights[n] * outputs[n][k] for n in range(N))
      <=
      sum(input_weights[m] * inputs[m][k] for m in range(M))
    )

  return model

def find_efficiency(alternative_nr: int, inputs_df: pd.DataFrame, outputs_df: pd.DataFrame):
  model = create_model(alternative_nr, inputs_ds, outputs_ds)
  model.solve(pulp.PULP_CBC_CMD(msg=False))
  return model.objective.value()

for k in range(len(inputs_ds)):
  print(f'Efficiency for alternative {k+1:0>2}: {find_efficiency(k, inputs_ds, outputs_ds)*100:.2f}%')


Efficiency for alternative 01: 100.00%
Efficiency for alternative 02: 100.00%
Efficiency for alternative 03: 59.12%
Efficiency for alternative 04: 100.00%
Efficiency for alternative 05: 79.98%
Efficiency for alternative 06: 30.00%
Efficiency for alternative 07: 100.00%
Efficiency for alternative 08: 27.08%
Efficiency for alternative 09: 100.00%
Efficiency for alternative 10: 40.92%
Efficiency for alternative 11: 25.85%


## Hipotetyczna jednostka porównawcza oraz potrzebne poprawki

Dla każdej z odnalezionych jednostek nieefektywnych odnaleziono hipotetyczną jednostkę porównawczą oraz poprawki potrzebne do osiągnięcia efektywności.


## Superefektywność

Do wyznaczenia superefektywności należy z modelu zdjąć ograniczenie efektywności dla kolejnych kryteriów.

\begin{align*}
\max & \sum_{n=1}^{N}\mu_n\cdot y_{no} & \\
s.t. & \sum\limits_{m=1}^{M}\nu_m\cdot x_{mo}=1  &  \\
 & \sum\limits_{n=1}^{N}\mu_n\cdot y_{nk}\leq\sum\limits_{m=1}^{M}\nu_n\cdot x_{mk}, & k=1,2,...,K,\;k\neq o \\
 & \mu_n\nu_m\geq0, & m=1,2,...,M\;, n=1,2,...,N \\
\end{align*}



In [124]:
import pulp
from pulp import LpVariable, LpProblem, LpMaximize


def create_model(alternative_nr: int, inputs_df: pd.DataFrame, outputs_df: pd.DataFrame):
  N, M, K = map(len, [outputs_df.columns[1:], inputs_df.columns[1:], inputs_df])
  o = alternative_nr

  model = LpProblem("CCR-input", LpMaximize)

  input_weights = LpVariable.dicts("input_weight", range(M), 0, None, cat='Continuous')
  output_weights = LpVariable.dicts("output_weight", range(N), 0, None, cat='Continuous')

  inputs = inputs_df.iloc[:, 1:].values.T
  outputs = outputs_df.iloc[:, 1:].values.T

  model.setObjective(sum(
    output_weights[n] * outputs[n][o] for n in range(N)
  ))

  model.addConstraint(sum(
    input_weights[m] * inputs[m][o] for m in range(M)
  ) == 1)

  for k in filter(lambda k: k != o, range(K)):
    model.addConstraint(
      sum(output_weights[n] * outputs[n][k] for n in range(N))
      <=
      sum(input_weights[m] * inputs[m][k] for m in range(M))
    )

  return model

def find_efficiency(alternative_nr: int, inputs_df: pd.DataFrame, outputs_df: pd.DataFrame):
  model = create_model(alternative_nr, inputs_ds, outputs_ds)
  model.solve(pulp.PULP_CBC_CMD(msg=False))
  return model.objective.value()

for k in range(len(inputs_ds)):
  print(f'Efficiency for alternative {k+1:0>2}: {find_efficiency(k, inputs_ds, outputs_ds)*100:.2f}%')


Efficiency for alternative 01: 227.79%
Efficiency for alternative 02: 112.38%
Efficiency for alternative 03: 59.12%
Efficiency for alternative 04: 103.99%
Efficiency for alternative 05: 79.98%
Efficiency for alternative 06: 30.00%
Efficiency for alternative 07: 200.00%
Efficiency for alternative 08: 27.08%
Efficiency for alternative 09: 174.59%
Efficiency for alternative 10: 40.92%
Efficiency for alternative 11: 25.85%


## Efektywność krzyżowa

## Rozkład efektywności

## Ranking jednostek
